In [2]:
import cv2
import depthai as dai
import numpy as np


In [3]:
img1 = cv2.imread('sruthi_video/16696818566204.png')
img2 = cv2.imread('sruthi_video/16696818566549.png')


In [5]:
def stitch_2_imgs(img1, img2, feature) : 
    img1_bw = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_bw = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    if feature == "SIFT":
        feat = cv2.SIFT_create() 
    elif feature == "ORB":
        feat = cv2.ORB_create()
    else :
        print("PLease enter correct feature value.")
        return 

    kp_img1, desc_img1 = feat.detectAndCompute(img1_bw, None) 
    kp_img2, desc_img2 = feat.detectAndCompute(img2_bw, None) 

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc_img2, desc_img1, k=2)

    good_points = []
    for m, n in matches: 
        if(m.distance < 0.6*n.distance): 
            good_points.append(m) 

    query_pts = np.float32([kp_img2[m.queryIdx] 
                    .pt for m in good_points]).reshape(-1, 1, 2) 

    train_pts = np.float32([kp_img1[m.trainIdx] 
                    .pt for m in good_points]).reshape(-1, 1, 2) 

    matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0) 

    dst = cv2.warpPerspective(img2, matrix, ((img1.shape[1] + img2.shape[1]), img2.shape[0])) 

    dst[0:img1.shape[0], 0:img1.shape[1]] = img1

    return dst

In [6]:
img1_2 = stitch_2_imgs(img1, img2, "SIFT")

cv2.imwrite(f"q4_stiched_frames.png", img1_2)


True